## **Import important Libraries**

In [ ]:
from matplotlib import image
from matplotlib import pyplot
import cv2 as cv
import glob
import io 
import math
import numpy as np
import os
import torch

## **Connect Colab to Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


## **Get YOLOv7 Model from Github**

In [ ]:
%cd /content/gdrive/MyDrive
!git clone https://github.com/augmentedstartups/yolov7.git
%cd yolov7
!pip install -r requirements.txt
!pip install roboflow

/content/gdrive/MyDrive
fatal: destination path 'yolov7' already exists and is not an empty directory.
/content/gdrive/MyDrive/yolov7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 49.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.0/49.0 KB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 14.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 KB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 KB 7.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9674 sha256=601fb9383fa5201479fe992d03a332bb77705ce29b5ec6ebfde10d689b4a5012
  Stored in directory: /root/.cache/pip/wheels/bd/a8/c3/3cf2c14a1837a4e04bd98631724e

## **Get traning, validation and test sets (from Roboflow)** - Do not share api_key

In [ ]:
%cd /content/gdrive/MyDrive/yolov7

#### ROBOFLOW DATASET DOWNLOAD CODE #####
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="")
project = rf.workspace("").project("")
dataset = project.version(1).download("")

/content/gdrive/MyDrive/yolov7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Centrosome-final-sum-proj-1 in yolov7pytorch:: 100%|██████████| 56/56 [00:22<00:00,  2.53it/s]


## **Start Training (uncomment last line)**

In [ ]:
%cd /content/gdrive/MyDrive/yolov7
!python train.py --batch 16 --cfg cfg/training/yolov7.yaml --epochs 1200 --data {dataset.location}/data.yaml --weights 'yolov7.pt' --device 0

/content/gdrive/MyDrive/yolov7
Traceback (most recent call last):
  File "train.py", line 12, in <module>
    import torch.distributed as dist
  File "/usr/local/lib/python3.8/dist-packages/torch/__init__.py", line 192, in <module>
    from torch._C import *  # noqa: F403
RuntimeError: KeyboardInterrupt: <EMPTY MESSAGE>


## **Define function to calculate Raw Integrated Density (RID) measure**

In [ ]:
def raw_integrated_density(img,xmin,xmax,ymin,ymax):
  sum = 0
  for line in range(len(img)):
    if line <= ymax and line >= ymin:
      for col in range(len(img[line])):
        if col <= xmax and col >= xmin:
          # print(img[line][col] )
          sum += img[line][col] 
  
  return sum 

## **Perform Detections on Test set using best model** 



*   We will obtain a new folder on Google Drive with the test images annotated with the objects detected 



Get the best.pt file here (link works for one week): https://wetransfer.com/downloads/786cc74ea3f19e0cc6a08a3bcd8e0a7420230216220820/4aff2f0b357bc5b522916ac1a03d197020230216220834/dce457?trk=TRN_TDL_01&utm_campaign=TRN_TDL_01&utm_medium=email&utm_source=sendgrid

In [ ]:
!python detect.py --weights /content/gdrive/MyDrive/yolov7/runs/train/exp6/weights/best.pt --conf 0.1 --source /content/gdrive/MyDrive/yolov7/Centrosome-final-sum-proj-1/test/images

## **Perform Detections on Test set using best model** 


*   After getting images with the objects detected for the entire dataset, we will now create a function the performs detection for an image only and exports it to a csv file; 
*   We will obtain their position on the image, confidence, class and some additional metrics like the center of mass of the objects, the area of the bounding boxes and the RID;



In [ ]:
#@title
def get_table_results(image_str):
  
  # Model
  model_name="runs/train/exp6/weights/best.pt"
  model = torch.hub.load(os.getcwd(), 'custom', model_name, source='local')  # local repo
  model.conf = 0.1

  files = [file for file in glob.glob("../yolov7/Centrosome-final-sum-proj-1/test/images/*")]

  for im in files:
    if image_str in im:
    
      # # Inference
      results = model(cv.cvtColor(cv.imread(im),cv.COLOR_BGR2RGB))

      # # Results3
      results.print()  # or .show(), .save(), .crop(), .pandas(), etc.

      results.xyxy[0]  # im predictions (tensor)
      results_table = results.pandas().xyxy[0]
      
      im_array = image.imread(im)
      im_array_sum = np.sum(im_array, axis = 2) 

      results_table = results_table.sort_values(by=['class'], ascending=True).reset_index(drop = True)

      results_table["id"] = results_table.index

      results_table["mass_center_x"] = (results_table["xmax"]+results_table["xmin"])/2
      
      results_table["mass_center_y"] = (results_table["ymax"]+results_table["ymin"])/2

      results_table["box_area"] = (results_table["xmax"]-results_table["xmin"])*(results_table["ymax"]-results_table["ymin"])
      
      results_table["RID_red"] = results_table.apply(lambda x: raw_integrated_density(im_array[:,:,0],
                                                                                  x.xmin,
                                                                                  x.xmax,
                                                                                  x.ymin,
                                                                                  x.ymax), axis=1)
      
      results_table["RID_green"] = results_table.apply(lambda x: raw_integrated_density(im_array[:,:,1],
                                                                                  x.xmin,
                                                                                  x.xmax,
                                                                                  x.ymin,
                                                                                  x.ymax), axis=1)
      
      results_table["RID_blue"] = results_table.apply(lambda x: raw_integrated_density(im_array[:,:,2],
                                                                                  x.xmin,
                                                                                  x.xmax,
                                                                                  x.ymin,
                                                                                  x.ymax), axis=1)
      
      results_table["RID_total"] = results_table.apply(lambda x: raw_integrated_density(im_array_sum,
                                                                                  x.xmin,
                                                                                  x.xmax,
                                                                                  x.ymin,
                                                                                  x.ymax), axis=1)
  table_name = "Table of Results - "+image_str+".csv"
  results_table.to_csv(table_name)
  return image_str, results_table


In [ ]:
image_str1, results_table1 = get_table_results("SUM_Composite_TRITC_GFP_DAPI_Position5_Zstack_magenta_png.rf.8be8653800c0f8ee3e47fb8c5b550989")

torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3190.)


Adding autoShape... 
image 1/1: 1200x1200 18 Cells, 23 Centrosomes
Speed: 25.4ms pre-process, 6452.3ms inference, 68.0ms NMS per image at shape (1, 3, 640, 640)


In [ ]:
image_str2, results_table2 = get_table_results("SUM_Composite_TRITC_GFP_DAPI_Position3_Zstack_magenta_png.rf.a82fe0eb91f5bd8f9d08f09ce7debffa")

Adding autoShape... 
image 1/1: 1200x1200 18 Cells, 19 Centrosomes
Speed: 3.6ms pre-process, 27.5ms inference, 1.6ms NMS per image at shape (1, 3, 640, 640)


In [ ]:
results_table1[["name","id","xmin","ymin","xmax","ymax","mass_center_x","mass_center_y","box_area"]]

,name,id,xmin,ymin,xmax,ymax,mass_center_x,mass_center_y,box_area
0,Cell,0,171.969269,846.871704,362.483124,986.930420,267.226196,916.901062,26683.125875
1,Cell,1,342.599335,77.584435,577.066589,254.550552,459.832962,166.067493,41492.759818
2,Cell,2,672.050598,754.559692,872.545349,927.216797,772.297974,840.888245,34616.843169
3,Cell,3,774.452393,282.967285,977.144897,529.559265,875.798645,406.263275,49982.346106
4,Cell,4,451.677948,551.391968,607.924133,804.513062,529.801041,677.952515,39549.205318
5,Cell,5,741.708069,530.227112,876.487427,710.598022,809.097748,620.412567,24310.275522
6,Cell,6,18.612333,651.912781,182.423737,833.597412,100.518035,742.755096,29762.014414
7,Cell,7,1023.403198,54.192154,1183.811157,253.595047,1103.607178,153.893600,31985.811092
8,Cell,8,150.594223,423.301361,400.336395,577.009460,275.465309,500.155411,38387.394627
9,Cell,9,598.250305,169.115280,770.184448,367.853882,684.217377,268.484581,34169.951175


In [ ]:
results_table2[["name","id","xmin","ymin","xmax","ymax","mass_center_x","mass_center_y","box_area"]]

,name,id,xmin,ymin,xmax,ymax,mass_center_x,mass_center_y,box_area
0,Cell,0,487.881409,56.576805,752.673889,288.088989,620.277649,172.332897,61302.685498
1,Cell,1,0.000000,168.495377,67.236076,472.103424,33.618038,320.299400,20413.413863
2,Cell,2,618.412903,1136.348389,862.893921,1199.561646,740.653412,1167.955017,15454.441387
3,Cell,3,22.081209,304.862061,175.585037,468.762909,98.833123,386.812485,25159.407648
4,Cell,4,344.182831,252.753510,548.797363,507.268829,446.490097,380.011169,52077.533172
5,Cell,5,672.183472,712.613342,826.680542,895.806458,749.432007,804.209900,28302.799605
6,Cell,6,1097.985840,370.535767,1199.515991,585.230103,1148.750916,477.882935,21797.948425
7,Cell,7,849.674927,219.837372,989.866943,406.728668,919.770935,313.283020,26200.667726
8,Cell,8,937.895752,3.218858,1085.261353,160.559769,1011.578552,81.889313,23186.637798
9,Cell,9,141.656952,2.459850,300.855988,139.295410,221.256470,70.877630,21784.089207


In [ ]:
def get_centrosomes_info(image_str, results_table):
  df_cell = results_table.copy()
  df_centrosome = results_table.copy()

  df_cell = df_cell[['id','class', 'name', 'mass_center_x', 'mass_center_y']][df_cell["name"]=="Cell"]
  df_centrosome = df_centrosome[['id','class', 'name', 'mass_center_x', 'mass_center_y',"RID_red","RID_green","RID_blue","confidence"]][df_centrosome["name"]=="Centrosome"]
  values = []
  for i in range(len(df_centrosome)):
    
    x_val = df_centrosome.iloc[i]["mass_center_x"]
    y_val = df_centrosome.iloc[i]["mass_center_y"]

    cell_id = -1
    dist = math.inf

    df_centrosome["nearest_cell"] = math.inf
    for j in range(len(df_cell)):
      new_dist = math.sqrt((x_val-df_cell.iloc[j]["mass_center_x"])**2+(y_val-df_cell.iloc[j]["mass_center_y"])**2)
    
      if new_dist < dist: 
        dist = new_dist
        cell_id = j
    values.append(cell_id)

  df_centrosome["nearest_cell"] = values
  table_name = "Centrosomes Information - "+image_str+".csv"
  df_centrosome.to_csv(table_name)
  return df_centrosome


In [ ]:
results_table_centrosomes1 = get_centrosomes_info(image_str1, results_table1)

In [ ]:
results_table_centrosomes1

,id,class,name,mass_center_x,mass_center_y,RID_red,RID_green,RID_blue,confidence,nearest_cell
18,18,1,Centrosome,1036.823792,153.850616,15005,12998,20051,0.342354,7
19,19,1,Centrosome,1006.989838,797.615601,15454,18055,30274,0.332879,12
20,20,1,Centrosome,680.780487,241.013084,11835,24092,39466,0.285837,9
21,21,1,Centrosome,351.593567,441.594650,23985,39046,44696,0.283514,11
22,22,1,Centrosome,779.220032,997.507477,18921,38467,79779,0.177916,15
23,23,1,Centrosome,856.700378,781.399902,15681,21838,22899,0.223701,2
24,24,1,Centrosome,140.935326,731.514832,13128,25042,50331,0.187263,6
25,25,1,Centrosome,856.659424,775.144012,39205,58773,62264,0.342493,2
26,26,1,Centrosome,589.243134,635.368591,14268,17574,26513,0.268892,4
27,27,1,Centrosome,468.123428,189.233124,26408,67481,136828,0.375475,1


In [ ]:
results_table_centrosomes2 = get_centrosomes_info(image_str2, results_table2)

In [ ]:
results_table_centrosomes2

,id,class,name,mass_center_x,mass_center_y,RID_red,RID_green,RID_blue,confidence,nearest_cell
18,18,1,Centrosome,116.692776,377.961319,13338,26367,54683,0.405089,3
19,19,1,Centrosome,1182.823669,458.791092,13645,27888,54735,0.294706,6
20,20,1,Centrosome,900.979279,327.148392,14207,39635,66385,0.296394,7
21,21,1,Centrosome,515.501648,356.220474,13750,12344,25143,0.305760,4
22,22,1,Centrosome,625.436005,137.066296,9850,18944,49735,0.313725,0
23,23,1,Centrosome,502.528336,114.174606,15861,20750,23030,0.364955,0
24,24,1,Centrosome,118.950184,545.735291,33181,42386,42347,0.386538,3
25,25,1,Centrosome,50.004532,456.384323,16623,19601,27652,0.456033,3
26,26,1,Centrosome,723.196411,771.373627,15131,20724,50169,0.702231,5
27,27,1,Centrosome,239.129517,993.786316,32838,49967,87179,0.503680,11
